<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=25237f7358136321033889633fdc7891bdc73440a7e120be2eaa3627aa0508a9
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-21 14:41:46
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.44 C
-------------------
Today PnL: -1.06 L (-0.73%)
Current PnL: -22.26 L (-14.59%)
CY Booked + Current PnL: -8.37 L (-5.48%)
-------------------
Total profit:  1.56 L
Total loss:  -23.82 L
-------------------
Total Booked + Current PnL: 18.61 L (14.81%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.68%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.67 L (61.08%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.64%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.31,-13.96,16.33,0.09,14160.0,-14068.0,86709.0,101.28,50.0,M-SC,4.16,253.0,-0.99,0.60,13.92,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.48,-8.54,9.35,0.02,14928.0,-14903.0,159656.0,143.30,73.0,M-SC,4.24,234.0,-1.00,1.11,38.87,OX40N,NTT,PAINTS
83,VOLTAS,1530.00,-1.36,8.94,9.87,19.69,20617.0,17148.0,208890.0,-1.63,53.0,X-MC,3.57,78.0,0.83,1.46,16.35,XY25,NTT,AC
43,ITC,452.00,0.60,-0.34,10.81,10.44,21562.0,-675.0,199463.0,-40.34,49.0,X-LC,0.96,5.0,-0.03,1.39,5.21,X40,NTT,FMCG
35,ICICIGI,2252.93,-1.06,7.56,11.75,20.19,22031.0,13177.0,187497.0,-15.78,53.0,X-MC,7.69,68.0,0.60,1.31,23.82,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.0,2.76,-37.08,111.19,32.89,54017.0,-28626.0,48581.0,-699.86,65.0,L-MC,5.40,259.0,-0.53,0.34,33.79,XR,NTT,BANKS
70,SURYODAY,216.0,2.75,-20.15,54.08,23.03,77331.0,-36077.0,142994.0,55.70,47.0,H-SC,11.66,167.0,-0.47,1.00,41.88,XR,NTT,BANKS
14,CAMPUS,393.0,1.38,-11.31,50.29,33.28,74829.0,-18981.0,148794.0,-28.59,37.0,M-SC,2.97,222.0,-0.25,1.04,15.33,XY24,NTT,FOOTWEAR
52,MEDANTA,1486.0,1.02,2.33,21.26,24.08,27617.0,2953.0,129903.0,-5.97,43.0,X-SC,4.51,89.0,0.11,0.91,22.40,XY24,NTT,HEALTHCARE
67,SIS,528.0,1.01,-25.05,61.79,21.26,51421.0,-27813.0,83219.0,1962.18,39.0,H-SC,3.30,166.0,-0.54,0.58,12.44,OX40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,STARHEALTH,761.00,-3.07,-5.17,55.29,47.26,147396.0,-14536.0,266587.0,29.34,48.0,H-SC,15.59,171.0,-0.10,1.86,42.83,XY24,NTT,INSURANCE
72,TANLA,1943.92,-3.05,-34.47,235.27,119.69,417414.0,-93342.0,177419.0,-33.88,31.0,H-SC,12.49,156.0,-0.22,1.24,40.16,AR,ATH,IT
56,QUESS,424.00,-2.77,-29.32,102.09,42.84,46908.0,-19058.0,45948.0,-53.72,36.0,X-SC,37.16,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
0,5PAISA,593.00,-2.27,-33.65,70.16,12.90,101715.0,-73520.0,144976.0,113.94,63.0,H-SC,15.04,161.0,-0.72,1.01,20.55,OX40N,NTT,FINANCE
32,HCLTECH,1922.01,-2.18,4.45,19.42,24.73,49073.0,10768.0,252692.0,8.94,61.0,X-LC,4.55,7.0,0.22,1.76,20.36,X40,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.94,0.79,105.11,106.72,130658.0,970.0,124306.0,-50.06,36.0,H-SC,4.42,139.0,0.01,0.87,24.35,AR,ATH,MISC
85,WIPRO,420.00,-0.34,0.66,71.13,72.27,121329.0,1126.0,170574.0,-13.79,54.0,M-LC,5.04,99.0,0.01,1.19,6.88,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.93,-3.70,115.91,107.91,164796.0,-5468.0,142176.0,-21.77,36.0,M-SC,10.19,216.0,-0.03,0.99,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.48,-8.54,9.35,0.02,14928.0,-14903.0,159656.0,143.30,73.0,M-SC,4.24,234.0,-1.00,1.11,38.87,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.31,-13.96,16.33,0.09,14160.0,-14068.0,86709.0,101.28,50.0,M-SC,4.16,253.0,-0.99,0.60,13.92,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.33,-21.04,43.70,13.47,93055.0,-56727.0,212940.0,-67.53,31.0,H-SC,1.67,158.0,-0.61,1.48,8.27,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,-0.72,-23.21,71.94,32.03,72826.0,-30603.0,101232.0,-16.88,47.0,H-SC,6.50,148.0,-0.42,0.71,13.39,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.34,0.66,71.13,72.27,121329.0,1126.0,170574.0,-13.79,54.0,M-LC,5.04,99.0,0.01,1.19,6.88,XR,NTT,IT
38,INDIAMART,4810.62,-1.94,0.79,105.11,106.72,130658.0,970.0,124306.0,-50.06,36.0,H-SC,4.42,139.0,0.01,0.87,24.35,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.94,0.79,105.11,106.72,130658.0,970.0,124306.0,-50.06,36.0,H-SC,4.42,139.0,0.01,0.87,24.35,AR,ATH,MISC
85,WIPRO,420.00,-0.34,0.66,71.13,72.27,121329.0,1126.0,170574.0,-13.79,54.0,M-LC,5.04,99.0,0.01,1.19,6.88,XR,NTT,IT
25,FINCABLES,1641.55,-0.93,-3.70,115.91,107.91,164796.0,-5468.0,142176.0,-21.77,36.0,M-SC,10.19,216.0,-0.03,0.99,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-0.18,-2.32,38.85,35.63,79531.0,-4862.0,204712.0,-16.30,32.0,H-MC,4.77,119.0,-0.06,1.43,13.97,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-0.67,-5.43,92.37,81.92,100277.0,-6238.0,108560.0,-44.55,31.0,M-SC,6.18,236.0,-0.06,0.76,28.52,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.06,-35.58,109.87,35.20,91073.0,-45778.0,82892.0,8.74,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-0.85,-15.54,69.69,43.32,95866.0,-25312.0,137560.0,-27.45,24.0,X-SC,1.69,90.0,-0.26,0.96,5.62,X40N,ATH,ELECTRICAL
76,TMPV,600.00,0.75,-14.10,65.52,42.18,155569.0,-38972.0,237438.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.75,XY24,NTT,AUTO
46,JSWINFRA,342.00,-1.92,-8.71,26.22,15.23,47954.0,-17456.0,182891.0,-21.89,25.0,X-MC,6.37,66.0,-0.36,1.27,20.73,X40N,NTT,REALTY
26,GILLETTE,11206.78,-1.62,-2.31,35.55,32.42,88173.0,-5871.0,248025.0,-19.73,29.0,X-SC,6.30,85.0,-0.07,1.73,12.44,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.75,-14.10,65.52,42.18,155569.0,-38972.0,237438.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.75,XY24,NTT,AUTO
43,ITC,452.00,0.60,-0.34,10.81,10.44,21562.0,-675.0,199463.0,-40.34,49.0,X-LC,0.96,5.0,-0.03,1.39,5.21,X40,NTT,FMCG
36,ICICIPRULI,790.00,-0.97,1.53,29.00,30.97,52036.0,2695.0,179433.0,-22.41,50.0,X-MC,1.50,75.0,0.05,1.25,14.28,X40,ATH,INSURANCE
34,HONAUT,58357.33,-0.85,-15.54,69.69,43.32,95866.0,-25312.0,137560.0,-27.45,24.0,X-SC,1.69,90.0,-0.26,0.96,5.62,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-0.67,-22.72,112.40,64.15,206704.0,-54051.0,183900.0,-55.00,47.0,X-SC,1.82,82.0,-0.26,1.28,3.16,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.06,-35.58,109.87,35.20,91073.0,-45778.0,82892.0,8.74,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.73,-13.60,71.83,48.45,124433.0,-27273.0,173233.0,-19.87,30.0,X-MC,7.29,64.0,-0.22,1.21,0.00,X40N,ATH,FINANCE
56,QUESS,424.00,-2.77,-29.32,102.09,42.84,46908.0,-19058.0,45948.0,-53.72,36.0,X-SC,37.16,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
55,PGHH,17907.65,-1.62,-4.26,39.71,33.76,76350.0,-8550.0,192270.0,-32.51,34.0,X-MC,3.05,57.0,-0.11,1.34,0.72,X40,ATH,FMCG
76,TMPV,600.00,0.75,-14.10,65.52,42.18,155569.0,-38972.0,237438.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.75,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-2.77,-29.32,102.09,42.84,46908.0,-19058.0,45948.0,-53.72,36.0,X-SC,37.16,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.88,-46.82,186.41,52.31,143895.0,-67967.0,77193.0,-43.37,31.0,X-SC,3.42,91.0,-0.47,0.54,3.33,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.06,-35.58,109.87,35.20,91073.0,-45778.0,82892.0,8.74,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,1.02,2.33,21.26,24.08,27617.0,2953.0,129903.0,-5.97,43.0,X-SC,4.51,89.0,0.11,0.91,22.40,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.97,-3.60,26.12,21.58,35288.0,-5040.0,135099.0,-51.38,36.0,X-MC,7.20,56.0,-0.14,0.94,19.61,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,0.47,-14.15,38.94,19.28,115649.0,-48963.0,296993.0,-25.24,64.0,X-LC,7.77,1.0,-0.42,2.07,9.79,X40,ATH,IT
41,INFY,2275.00,0.88,5.52,46.77,54.87,154411.0,17262.0,330150.0,-17.80,61.0,X-LC,3.31,2.0,0.11,2.30,14.26,X40,BTT,IT
76,TMPV,600.00,0.75,-14.10,65.52,42.18,155569.0,-38972.0,237438.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.65,0.75,XY24,NTT,AUTO
82,VBL,671.64,-1.09,-9.79,50.39,35.67,143577.0,-30911.0,284931.0,-20.46,40.0,X-LC,6.13,4.0,-0.22,1.99,2.71,X40N,ATH,FMCG
43,ITC,452.00,0.60,-0.34,10.81,10.44,21562.0,-675.0,199463.0,-40.34,49.0,X-LC,0.96,5.0,-0.03,1.39,5.21,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.44,-12.56,107.76,81.67,88394.0,-11781.0,82029.0,7377.78,53.0,L-SC,18.40,271.0,-0.13,0.57,58.85,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.10,-12.27,59.05,39.54,52407.0,-12408.0,88751.0,-31.43,70.0,M-SC,6.90,220.0,-0.24,0.62,25.15,AR,ATH,AUTO
51,MASFIN,398.61,0.03,-2.50,25.17,22.05,24046.0,-2445.0,95535.0,-15.99,59.0,H-SC,7.08,164.0,-0.10,0.67,38.87,XR,ATH,FINANCE
70,SURYODAY,216.00,2.75,-20.15,54.08,23.03,77331.0,-36077.0,142994.0,55.70,47.0,H-SC,11.66,167.0,-0.47,1.00,41.88,XR,NTT,BANKS
65,SHALBY,327.00,-1.67,-16.41,49.66,25.10,75195.0,-29723.0,151420.0,1013.24,33.0,M-SC,16.65,250.0,-0.40,1.05,21.35,XY24,NTT,HEALTHCARE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-1.48,-8.54,9.35,0.02,14928.0,-14903.0,159656.0,143.30,73.0,M-SC,4.24,234.0,-1.00,1.11,38.87,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.94,0.90,19.87,20.95,50476.0,2262.0,254030.0,-0.07,77.0,X-LC,12.14,31.0,0.04,1.77,36.85,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,2.76,-37.08,111.19,32.89,54017.0,-28626.0,48581.0,-699.86,65.0,L-MC,5.40,259.0,-0.53,0.34,33.79,XR,NTT,BANKS
0,5PAISA,593.00,-2.27,-33.65,70.16,12.90,101715.0,-73520.0,144976.0,113.94,63.0,H-SC,15.04,161.0,-0.72,1.01,20.55,OX40N,NTT,FINANCE
68,SONACOMS,806.63,-0.10,-12.27,59.05,39.54,52407.0,-12408.0,88751.0,-31.43,70.0,M-SC,6.90,220.0,-0.24,0.62,25.15,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,44.57
2,50,75.34


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.70
MC    29.24
LC    25.10
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.91
X40      21.10
X40N     11.86
XR        9.66
XY25      9.12
AR        9.01
OX40N     7.72
X200      1.86
SR        1.00
MH        0.80
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.08
X-MC    22.82
X-LC    19.96
M-SC    12.06
X-SC     8.15
H-MC     4.71
H-LC     2.97
L-SC     1.41
M-MC     1.37
M-LC     1.19
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.74,-4.81,38.70
IT,13.77,-14.42,73.49
FINANCE,9.46,-14.19,62.44
MISC,7.46,-21.40,71.13
PAINTS,6.00,-7.13,23.04
ELECTRICAL,5.88,-11.25,52.00
INSURANCE,4.42,0.21,34.96
PHARMA,3.85,-1.21,33.60
AUTO,3.34,-19.68,70.99


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3029999.0,21
AR,1280918.0,10
XR,1278747.0,13
X40,1010145.0,14
X40N,747320.0,9
OX40N,683502.0,10
XY25,332813.0,6
SR,276766.0,2
MH,69507.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3525402.0,25
M-SC,1327005.0,15
X-MC,1208225.0,16
X-LC,898068.0,11
X-SC,751590.0,8
H-MC,403218.0,3
L-SC,259610.0,3
H-LC,131496.0,2
M-LC,121329.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1212936.0      6
           AR         868842.0      5
           XR         782868.0      7
M-SC       XY24       774722.0      6
X-MC       X40        439728.0      7
X-LC       X40        391171.0      5
X-MC       XY24       347671.0      3
H-SC       OX40N      314483.0      4
X-SC       X40N       291115.0      3
M-SC       OX40N      281324.0      5
X-SC       XY24       281229.0      3
H-SC       SR         276766.0      2
X-MC       X40N       245560.0      4
X-LC       XY24       225789.0      2
H-MC       AR         215566.0      2
X-LC       X40N       210645.0      2
H-MC       XY24       187652.0      1
X-SC       X40        179246.0      2
X-MC       XY25       175266.0      2
L-SC       XR         171915.0      2
M-SC       XR         148618.0      2
           AR         122341.0      2
M-LC       XR         121329.0      1
L-SC       OX40N       87695.0      1
H-LC       AR          74169.0      1
X-LC       XY25        70463.0      2
H-SC       MH          69507.0      1
H-LC       X200        57327.0      1
L-MC       XR          54017.0      1
M-MC       XY25        47986.0      1
L-LC       XY25        39098.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
